In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from keras_tuner import HyperModel
from keras_tuner import RandomSearch

In [5]:
movies        = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/movies.csv')
links         = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/links.csv')
ratings       = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/ratings.csv')
tags          = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/tags.csv')

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [8]:
movies.isnull().sum()

,0
movieId,0
title,0
genres,0


In [9]:
print(movies['genres'].nunique())
print(movies['genres'].unique())

951
['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Drama' 'Action|Thriller'
 'Drama|Horror|Thriller' 'Comedy|Horror|Roman

In [10]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [11]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [12]:
links.isnull().sum() # There is some missing values in links dataframe

,0
movieId,0
imdbId,0
tmdbId,8


In [13]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [14]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [15]:
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [16]:
ratings['userId'].nunique()

610

In [17]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [18]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [19]:
tags.isnull().sum()

,0
userId,0
movieId,0
tag,0
timestamp,0


In [20]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [21]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

# Initial Model Setup and Baseline Performance

## Transoformer Model

In [22]:
# Encode userId and movieId as integers
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

ratings['userId'] = user_encoder.fit_transform(ratings['userId'])
ratings['movieId'] = movie_encoder.fit_transform(ratings['movieId'])

# Get the number of users and movies
num_users = ratings['userId'].nunique()
num_movies = ratings['movieId'].nunique()

# Prepare input features
X = ratings[['userId', 'movieId']]
y = ratings['rating'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Transformer-based recommendation model
class TransformerRecommender(Model):
    def __init__(self, num_users, num_movies, embed_dim, num_heads, ff_dim, num_transformer_blocks, rate=0.1):
        super(TransformerRecommender, self).__init__()

        # Embedding layers
        self.user_embedding = Embedding(num_users, embed_dim)
        self.movie_embedding = Embedding(num_movies, embed_dim)

        # Transformer layers
        self.transformer_blocks = [
            [
                MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim),
                LayerNormalization(epsilon=1e-6),
                Dense(ff_dim, activation="relu"),
                Dense(embed_dim),
                Dropout(rate)
            ]
            for _ in range(num_transformer_blocks)
        ]

        # Dense output layers
        self.global_pooling = GlobalAveragePooling1D()
        self.dropout = Dropout(rate)
        self.dense1 = Dense(ff_dim, activation='relu')
        self.dense2 = Dense(1, activation='linear')

    def call(self, inputs):
        user_id, movie_id = inputs

        # Embedding lookup
        user_emb = self.user_embedding(user_id)
        movie_emb = self.movie_embedding(movie_id)

        # Stack user and movie embeddings to create a sequence
        x = tf.stack([user_emb, movie_emb], axis=1)

        # Pass through transformer blocks
        for mha, ln, ff1, ff2, dropout in self.transformer_blocks:
            attn_output = mha(x, x)
            x = ln(x + attn_output)
            ff_output = ff1(x)
            ff_output = ff2(ff_output)
            x = ln(x + dropout(ff_output))

        # Global pooling and dense layers
        x = self.global_pooling(x)
        x = self.dropout(x)
        x = self.dense1(x)
        output = self.dense2(x)

        return output

# Model hyperparameters
embed_dim = 32
num_heads = 4
ff_dim = 128
num_transformer_blocks = 2
dropout_rate = 0.1

# Instantiate and compile the model
model = TransformerRecommender(num_users=num_users, num_movies=num_movies, embed_dim=embed_dim,
                               num_heads=num_heads, ff_dim=ff_dim, num_transformer_blocks=num_transformer_blocks,
                               rate=dropout_rate)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model summary
model.build([(None,), (None,)])  # Fixes the build issue
model.summary()

# Prepare input data for training
train_user_input = X_train['userId'].values
train_movie_input = X_train['movieId'].values
test_user_input = X_test['userId'].values
test_movie_input = X_test['movieId'].values

# Start timer for training time
start_time = time.time()

# Train the model
history = model.fit([train_user_input, train_movie_input], y_train, epochs=10, batch_size=64,
                    validation_data=([test_user_input, test_movie_input], y_test))

# End timer for training time
training_time = time.time() - start_time

# Predict for a specific user
user_id = 1

# Get the movies that user 1 has already rated
rated_movie_ids = ratings[ratings['userId'] == user_id]['movieId'].values

# Find the movies that user 1 has not rated
unrated_movie_ids = np.setdiff1d(np.array(range(num_movies)), rated_movie_ids)

# Prepare the input for prediction
user_input = np.full_like(unrated_movie_ids, user_id)
movie_input = unrated_movie_ids

# Predict ratings for all unrated movies
predicted_ratings = model.predict([user_input, movie_input]).flatten()

# Clip the predicted ratings to be within the range [1, 5]
predicted_ratings = np.clip(predicted_ratings, 1, 5)

# Get the top 10 movie indices
top_10_indices = np.argsort(predicted_ratings)[-10:][::-1]

# Get the top 10 movieIds and their predicted ratings
top_10_movie_ids = unrated_movie_ids[top_10_indices]
top_10_ratings = predicted_ratings[top_10_indices]

# Display the top 10 recommendations
recommended_movies = pd.DataFrame({
    'movieId': top_10_movie_ids,
    'predicted_rating': top_10_ratings
})

# Merge with the movies dataframe to get the movie titles
recommended_movies = pd.merge(recommended_movies, movies, on='movieId')

print("Top 10 movie recommendations for userId 1:")
print(recommended_movies[['movieId', 'title', 'predicted_rating']])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'transformer_recommender', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "transformer_recommender"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ multi_head_attention                 │ ?                           │     0 (unbuilt) │
│ (MultiHeadAttention)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization                  │ ?                           │     0 (unbuilt) │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ multi_head_attention_1               │ ?                           │     0 (unbuilt) │
│ (MultiHeadAttention)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_1                │ ?                           │     0 (unbuilt) │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - loss: 1.0534 - mean_absolute_error: 0.7804 - val_loss: 0.7920 - val_mean_absolute_error: 0.6988
Epoch 2/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6812 - mean_absolute_error: 0.6327 - val_loss: 0.7667 - val_mean_absolute_error: 0.6673
Epoch 3/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.6267 - mean_absolute_error: 0.6042 - val_loss: 0.7756 - val_mean_absolute_error: 0.6795
Epoch 4/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6003 - mean_absolute_error: 0.5883 - val_loss: 0.8352 - val_mean_absolute_error: 0.7030
Epoch 5/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.5656 - mean_absolute_error: 0.5677 - val_loss: 0.7901 - val_mean_absolute_error: 0.6801
Epoch 6/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.5189 - mean_absolute_error: 0.5426 - val_loss: 0.8189 - val_mean_absolute_error: 0.6799
Epoch 7/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.4836 - mean_absolute_e

In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Use the predictions from the model
y_pred = model.predict([test_user_input, test_movie_input]).flatten()

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print(f"Training Time: {training_time:.2f} seconds")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")


631/631 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
Training Time: 92.70 seconds
Mean Squared Error (MSE): 0.8454970528300847
Root Mean Squared Error (RMSE): 0.9195091368932038
Mean Absolute Error (MAE): 0.6877896764072952


In [24]:
import keras_tuner as kt

# Define a HyperModel class
class TransformerHyperModel(kt.HyperModel):
    def __init__(self, num_users, num_movies):
        self.num_users = num_users
        self.num_movies = num_movies

    def build(self, hp):
        embed_dim = hp.Int('embed_dim', min_value=16, max_value=64, step=16)
        num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2)
        ff_dim = hp.Int('ff_dim', min_value=64, max_value=256, step=64)
        num_transformer_blocks = hp.Int('num_transformer_blocks', min_value=1, max_value=3, step=1)
        dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)

        model = TransformerRecommender(
            num_users=self.num_users,
            num_movies=self.num_movies,
            embed_dim=embed_dim,
            num_heads=num_heads,
            ff_dim=ff_dim,
            num_transformer_blocks=num_transformer_blocks,
            rate=dropout_rate
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='mean_squared_error',
            metrics=['mean_absolute_error']
        )

        return model

# Instantiate the hypermodel
hypermodel = TransformerHyperModel(num_users=num_users, num_movies=num_movies)

# Set up the tuner
tuner = kt.RandomSearch(
    hypermodel,
    objective='val_mean_absolute_error',  # Minimize validation mean absolute error
    max_trials=10,  # Number of hyperparameter sets to try
    executions_per_trial=1,  # Number of times to train the model with the same hyperparameter set
    directory='transformer_tuning',
    project_name='movie_recommender'
)

# Define a callback to stop training when a good model is found
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Run the hyperparameter search
tuner.search([train_user_input, train_movie_input], y_train,
             validation_data=([test_user_input, test_movie_input], y_test),
             epochs=10,
             batch_size=64,
             callbacks=[early_stopping])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
best_model.summary()

# Predict on the test set
y_pred = best_model.predict([test_user_input, test_movie_input]).flatten()

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print(f"Best Model Mean Squared Error (MSE): {mse}")
print(f"Best Model Root Mean Squared Error (RMSE): {rmse}")
print(f"Best Model Mean Absolute Error (MAE): {mae}")


Trial 10 Complete [00h 00m 39s]
val_mean_absolute_error: 0.6665499210357666

Best val_mean_absolute_error So Far: 0.6660526394844055
Total elapsed time: 00h 09m 14s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 70 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "transformer_recommender"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 32)                  │          19,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 32)                  │         311,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ multi_head_attention                 │ (None, 2, 32)               │          25,184 │
│ (MultiHeadAttention)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization                  │ (None, 2, 32)               │              64 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2, 128)              │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2, 32)               │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2, 32)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ multi_head_attention_1               │ (None, 2, 32)               │          25,184 │
│ (MultiHeadAttention)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_1                │ (None, 2, 32)               │              64 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2, 128)              │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2, 32)               │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 2, 32)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 32)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 402,241 (1.53 MB)

 Trainable params: 402,241 (1.53 MB)

 Non-trainable params: 0 (0.00 B)

631/631 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
Best Model Mean Squared Error (MSE): 0.7732881590875748
Best Model Root Mean Squared Error (RMSE): 0.8793680452959243
Best Model Mean Absolute Error (MAE): 0.6660528654370786
